In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pylab as plt

# Fix the random seed to facilitate grading
np.random.seed(1)

# HW2.1 - Solvers for constrained optimization

## 1.a Optimality conditions (25 pts)

We start by creating a similar problem as in HW3-1, but adding linear constraints to it. We will see in the next notebook how we can solve a sequence of such programs to solve real-world problems such as the inverse kinematics problem of Practical 2. 

Our example will be

$$
\begin{aligned}
\min \, & x^\top Q x \\
\text{s.t.}\, & A x = b
\end{aligned}
$$


**1.a.1** Below's code is broken. Using the generated plot, explain in a couple of sentences why there must be a bug in your code. (4 pts)

In [ ]:
d = 2

Q = np.random.rand(d, d)
Q = Q.T @ Q

A = np.full((1, d), 0.2)
b = np.ones((1, 1))

def f(x, Q):
    return 0.5 * x.T @ Q @ x

def g(x, A, b):
    return A @ x - b

def f_grad(x, Q):
    return Q @ x

def g_jac(x, A, b):
    return A

print("A =", A)
print("b =", b)
print("Q =", Q)

In [ ]:
def setup_plot(Q, A, b, title=""):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 10)

    x = np.linspace(-10, 10, 200)
    y = np.linspace(-14, 6, 200)
    xx, yy = np.meshgrid(x, y)
    zz = np.array([f(np.array([xi, yi]), Q) for xi, yi in zip(xx.flatten(), yy.flatten())])
    
    ax.contourf(xx, yy, zz.reshape(xx.shape), levels=30)

    for a_i, b_i in zip(A, b):
        a_x, a_y = a_i
        ys = 1 / a_y * (b_i - a_x * x)
        ax.plot(x, ys, color="C1", label="g(x)=0")
        
    ax.set_title(title)
    ax.axis("equal")
    ax.set_ylim([-14, 6])
    return fig, ax

KKT_mat = np.hstack([np.vstack([Q, A]), np.vstack([A.T, np.zeros((A.shape[0], A.shape[0]))])])
KKT_vec = np.vstack([np.zeros((Q.shape[0], 1)), -b]) 
z_opt = np.linalg.solve(KKT_mat, KKT_vec)
x_opt, l_opt = z_opt[:d], z_opt[d:]
print(x_opt)
fig, ax = setup_plot(Q, A, b)
ax.scatter(x_opt[0], x_opt[1], color="white", marker="x", label="optimum")
plt.legend()

**Answer**:


**1.a.2** Using your observations above, implement corresponding unit tests and make sure that they fail. (6 pts)

In [ ]:
### ============= 1.a.2 Fill in below ==============
### Find the expression of the optimal solution using the Lagrangian
### and plot it. Explain below in words why this is an optimal solution. 
### Note that you can use the following structure to assert a test fails:
### try:
###    test_your_code()
### except AssertionError as e:
###    print("xx test failed!")
### else:
###    print("xx test passed!")

def test_feasibility(x_opt, A, b):
    NotImplementedError
    
def test_stationarity(x_opt, l_opt, Q, A, b):
    NotImplementedError
   
def test_feasible_cost(x_opt, Q, A, b):
    NotImplementedError

# test the candidate solution
try:
    test_feasible_cost(x_opt, Q, A, b)
except AssertionError as e:
    print("cost test failed!")
else:
    print("cost test passed!")

try:
    test_feasibility(x_opt, A, b)
except AssertionError as e:
    print("feasibility test failed!")
else:
    print("feasibility test passed!")
    
try:
    test_stationarity(x_opt, l_opt, Q=Q, A=A, b=b)
except AssertionError as e:
    print("stationarity test failed!")
else:
    print("stationarity test passed!")
### ===========================

**1.a.3** Now, fix the code and show that the tests pass. (5 pts)

In [ ]:
### ============= 1.a.3 Fill in below ==============
### Copy over the code from 1.a.1 and fix it. Show that the tests pass. 

x_opt, l_opt = None, None

fig, ax = setup_plot(Q, A, b)
ax.scatter(x_opt[0], x_opt[1], color="white", marker="x")
plt.legend()

# test the candidate solution
test_feasible_cost(x_opt, Q, A, b)
test_feasibility(x_opt, A, b)
test_stationarity(x_opt, l_opt, Q, A, b)
### ===========================

**1.a.4** Temporarily change A and b to break your algorithm. Can you think of two different ways to break it? Copy the below cell to create two different failure cases. (4 pts)

In [ ]:
### ============= 1.a.4 ====================
### Intentionally break your algorithm below. Make a copy of this cell
### and create another failure case. Comment the behavior (you can simply 
### change the failure message, or you can comment below the cell). 

Afail = None
bfail = None
failure_message = "put your failure message here"

### =================================

try:
    KKT_mat = np.hstack([np.vstack([Q, Afail]), np.vstack([Afail.T, np.zeros((Afail.shape[0], Afail.shape[0]))])])
    KKT_vec = np.vstack([np.zeros((Q.shape[0], 1)), bfail]) 
    z_opt = np.linalg.solve(KKT_mat, KKT_vec)
    x_opt, l_opt = z_opt[:d], z_opt[d:]
except Exception as e: 
    print(failure_message)
    print("Exception:", e)

**1.a.5** Finally, we change Q to the following. Do we still get the correct minimum? Comment on the behavior and what would be the correct solution? Is there a particular value for A that will change this behavior?  (6 pts)

In [ ]:
Q_new = np.zeros((d, d))
Q_new[0, 0] = -1.0
Q_new[1, 1] = -1e-10

KKT_mat = np.hstack([np.vstack([Q_new, A]), np.vstack([A.T, np.zeros((A.shape[0], A.shape[0]))])])
KKT_vec = np.vstack([np.zeros((Q_new.shape[0], 1)), b]) 
z_opt = np.linalg.solve(KKT_mat, KKT_vec)
x_opt, l_opt = z_opt[:d], z_opt[d:]

fig, ax = setup_plot(Q_new, A, b)
ax.scatter(x_opt[0], x_opt[1], color="white", marker="x")
plt.legend()

**Answer**:



## 1.b Newton solver and convergence (26 pts)

In practice, we usually don't have the globally optimal solution and so instead of finding the optimal solution in one shot, we use an iterative method. 


**1.b.1** (10 pts) Adopt your code from above to compute Newton updates `p_k, delta_k`, so that we can use the update equations:
$$
x_{k+1} = x_k + \alpha p_k, \quad \lambda_{k+1} = \lambda_k + \alpha \Delta_k
$$

Using this update, implement a simple damped Newton solver. In particular, 
- Use the following convergence criterion: $|| \nabla L(x, \lambda) ||_p \leq \epsilon$, where you can choose $p=\infty$ or $p=2$. 
- Use the unit tests from above to ensure that your code runs correctly.
- (optional): use the 2D plots from above to plot the convergence of the iterates; in particular, plot also the constraint gradients scaled by $\lambda$, and if you want, the constraints of $f$. Observe the convergence.

Discuss the convergence, in particular the final primal and dual residuals. 

**Answer**: 


In [ ]:

### ============= 1.b.1 Fill in below ==============
### Create a damped  Newton solver for the above problem. 
def damped_newton_eq_qp(
    x0, Q, A, b, tol=1e-10, max_iter=50, alpha=0.5, infeasible=False
):

    return x_opt, l_opt, res_pri_list, res_dual_list, x_list, l_list
### =========================== 

# ---- run solver ----
x_0 = np.array([0, 5])
x_opt, l_opt, res_pri_list, res_dual_list, x_list, l_list = damped_newton_eq_qp(x_0, Q, A, b)
print("x_opt:", x_opt)
print("l_opt:", l_opt)
print("final residuals:", res_pri_list[-1], res_dual_list[-1])


### ============= 1.b.1 Fill in below ==============
### Test and plot output

test_feasibility(x_opt, A, b)
test_stationarity(x_opt, l_opt, Q, A, b)
test_feasible_cost(x_opt, Q, A, b)

### =========================== 

**1.b.2** Adopt the functions from homework 1.3 to plot the convergence of your solver. 
For the left plot, instead of plotting the cost, plot the primal and dual residuals. For the right plot, you can still plot the log-log distance to the optimum at steps k vs. k+1. Then, answer the following questions: (8 pts)
a) Why do you think do we plot the gradients now, but we didn't plot them for the neural network solvers?
b) What is the convergence rate and why? 

**Answers**:


In [ ]:
### ============= 1.b.2 Fill in below ==============

# solution:
def plot_convergence_rate(res_pri_list, res_dual_list, x_list, l_list):    
    raise NotImplementedError

### ===========================

plot_convergence_rate(res_pri_list, res_dual_list, x_list, l_list)

**1.b.3** Start your solver at an infeasible point and observe the convergence. If you haven't done so yet, make sure to adapt your code so that it works for infeasible starts. Comment on the behavior. Is the convergence as you expected? Does the solver favor feasibility or optimality? Commong on the dual residual convergence and on the convergence rate? (8 pts)

**Answer**: 


In [ ]:
x_0 = np.array([-5, 0])

### ============= 1.b.3 Fill in below ==============


### ===========================

fig, ax = setup_plot(Q, A, b)
ax.scatter(x_opt[0], x_opt[1], color="white", marker="x")
for x_i, l_i in zip(x_list, l_list):
    ax.scatter(x_i[0], x_i[1], color="white", marker="o") 
plt.legend()


plot_convergence_rate(res_pri_list, res_dual_list, x_list, l_list)

## 1.c Penalty and Augmented Lagrangian (16 pts)

Now, let's see how the convergence changes when we use the Penalty or Augmented Lagrangian method instead. 

**1.c.1** Below you can find an implementation of the Quadratic Penalty Method. Discuss the behavior for the following parameters. (6 pts)
- Can you explain the difference in behavior of x_opt and x_approx? 
- What happens when you set max_iter to 1000 and why?
- What happens when you set rho to 100 and why?

**Answer**:


In [ ]:
import pandas as pd
import seaborn as sns

x_opt = np.zeros(d)
x_approx = np.zeros(d)

alpha = 0.01
eps = 0.05

rho = 5.0

data = []

max_iter = 100
max_iter_inner = 100
for i in range(max_iter):    
    # optimal solution
    x_opt = np.linalg.solve(Q + rho *A.T @ A, rho * np.sum(A.T @ b, axis=1))
        
    # approximate argmin
    for i_inner in range(max_iter_inner):
        grad = f_grad(x_approx, Q) + rho * np.sum(g_jac(x_approx, A, b).T @ g(x_approx, A, b), axis=1)
        cost = f(x_approx, Q) + rho / 2 * np.sum(g(x_approx, A, b)**2)
        x_approx = x_approx - alpha * grad

    data.append(
        {
            "rho": rho,
            "cost": f(x_approx, Q),
            "dual residual": np.max(np.abs(A @ x_approx - b)), 
            "iter": i,
            "method": "approx",
        }
    )
    data.append(
        {
            "rho": rho,
            "cost": f(x_opt, Q),
            "dual residual": np.max(np.abs(A @ x_opt - b)), 
            "iter": i,
            "method": "opt",
        }
    )
    # update rho
    rho = (1 + eps) * rho
    
### ===========================
df = pd.DataFrame(data)
fig, ax = plt.subplots()
sns.lineplot(df, ax=ax, x="iter", y="dual residual", hue="method")
ax.set_yscale("log")

We see above that convergence is a bit finicky with the Penalty method. Next we will study if things get better when we use the Augmented Lagrangian instead. 

**I.c.2** Implement the Augmented Lagrangian method below. (10 pts)

In [ ]:
import pandas as pd
import seaborn as sns

### ============= 1.c.2 Fill in below ==============

x_opt = np.zeros(d)
l_opt = np.zeros(1)
x_approx = np.zeros(d)
l_approx = np.zeros(1)

alpha = 0.01
rho = 5.0

data = []

max_iter = 100
max_iter_inner = 100
for i in range(max_iter):    

    x_opt, l_opt = None, None
    
    # approximate argmin
    x_approx = None
    l_approx = None

    data.append(
        {
            "rho": rho,
            "cost": f(x_approx, Q),
            "dual residual": np.max(np.abs(A @ x_approx - b)), 
            "iter": i,
            "method": "approx",
        }
    )
    data.append(
        {
            "rho": rho,
            "cost": f(x_opt, Q),
            "dual residual": np.max(np.abs(A @ x_opt - b)), 
            "iter": i,
            "method": "opt",
        }
    )
    
### ===========================
df = pd.DataFrame(data)
fig, ax = plt.subplots()
sns.lineplot(df, ax=ax, x="iter", y="dual residual", hue="method")
ax.set_yscale("log")

## 1.d ADMM (16 pts)

We consider $\ell_1$ penalized linear regression, also known as the LASSO problem: [Wikipedia](https://en.wikipedia.org/wiki/Lasso_(statistics)). Given an output vector $b \in \mathbb{R}^n$, a matrix $A \in \mathbb{R}^{n \times p}$ of predictor variables, and a tuning parameter $\lambda \ge 0$, the lasso estimate can be defined as

$$
\min_{x \in \mathbb{R}^n} \; \frac{1}{2}\|Ax - b\|_2^2 + \lambda \|x\|_1
$$

The first term $\|Ax - b\|_2^2$ measures how well the model fits the data, while the second term $\|x\|_1$ penalizes the absolute values of the coefficients. The $\ell_1$ penalty encourages **sparsity**, meaning that many components of $x$ become exactly zero. As a result, LASSO performs both **regression** and **feature selection** simultaneously.

Because the $\ell_1$ norm is **non-smooth**, the objective function is no longer a standard quadratic program (QP), and the closed-form solution for least squares do not apply. In this question, we use the **Alternating Direction Method of Multipliers (ADMM)**, which is well suited for problems with non-smooth regularization terms and allows the optimization to be decomposed into simpler subproblems.

We start by rewriting the problem using variable splitting (ADMM form)

$$
\begin{aligned}
\min_{x,z}\quad & \frac{1}{2}\|Ax-b\|_2^2 + \lambda\|z\|_1 \\
\text{s.t.}\quad & x = z
\end{aligned}
$$

**I.d.1** (16 pts) Implement the ADMM method below. In particular: 

- Use the convergence critera as discussed in [Boyd Ch. 3.3.1](https://canvas.cmu.edu/courses/51076/files/folder/04_Resources?preview=13869096)
- Try the automatic update rule of $\rho$ as discussed in [Boyd Ch. 3.4.1](https://canvas.cmu.edu/courses/51076/files/folder/04_Resources?preview=13869096)

Discuss the performance: 

a) Why is it interesting to use ADMM here? 

b) Discuss the observed convergence rate.

c) Does the automatic update rule for $\rho$ help? 

**Anwers**: 



In [ ]:
import numpy as np

### ============= 1.d.1 Implement ADMM below ==============

    
### ===========================

# 1.e Packaging of functions (10 pts)

Just like in Homework 1, you will package what you wrote above as an easy-to-use python package for the next practical. 

**1.e.1** Create a python package, for example ``myoptim''. Move the relevant functions to this package, and create appropriate tests. After doing this, you should be able to install the package using `pip install -e myoptim`. Then make sure you create some tests and show their outputs, as in the example below. (5 pts)

List of solvers for QP problem that need to move to this package:
- Damped Newton solver 
- Augmented Lagrangian solver

In [ ]:
!pytest myoptim/tests

**1.e.2** Answer the questions below about the package that you created. Note that there is no right or wrong; these questions serve to make you reflect about what you have implemented. (5 pts)
- What tests did you implement? Explain each test with one short sentence.
- Are you sure that your code works based on these tests? Reflect on whether all crucial aspects of the code are tested. What could still go wrong? 
- Is the interface to your code straightforward (i.e., how many lines of code are required to run the solvers? Would it be easy for someone to use it? A good sanity check is if your code is modular so that many functions can be easily tested.) 

## Acknowledgment of Collaboration and/or Tool Use

Please choose from below (simply delete the lines that do not apply) and add a few additional notes

- “I worked alone on this assignment.”, or
- “I worked with ~~~~~~ [person or tool] on this assignment.” and/or
- “I received assistance from ~~~~~~ [person or tool] on this assignment.”

For the last two cases, specify how the person or tool helped you and explain why this amplified your learning process:

_add answer here_